In [1]:
from dotenv import load_dotenv
import os
import time
from langchain_community.utilities import SQLDatabase

load_dotenv()

True

## 1. vectorstore 구현


In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import datetime

In [3]:
embedding = OpenAIEmbeddings()
vector_store = Chroma(persist_directory="../data/chroma_db", embedding_function=embedding)

In [4]:
## 더미데이터 저장

dummy_conversations = [
    {
        "session_id": "1",
        "role": "user",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "안녕하세요! 아메리카노 주문할 수 있나요?"
    },
    {
        "session_id": "1",
        "role": "assistant",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "네, 가능합니다! ICE로 하시겠어요? HOT으로 하시겠어요?"
    },
    {
        "session_id": "1",
        "role": "user",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "아이스로 한 잔 주세요!"
    },
    {
        "session_id": "1",
        "role": "assistant",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "주문이 완료되었습니다! 결제 방법을 선택해주세요."
    },
    {
        "session_id": "2",
        "role": "user",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "카페라떼 가격이 얼마인가요?"
    },
    {
        "session_id": "2",
        "role": "assistant",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "카페라떼는 2,900원입니다."
    }
]

def store_data(data_list):
    """더미 데이터를 ChromaDB에 저장"""
    documents = [
        Document(
            page_content=item["content"] if "content" in item else f"주문 내역: {item['items']}",
            metadata=item
        )
        for item in data_list
    ]
    vector_store.add_documents(documents)

# 데이터 저장 실행
store_data(dummy_conversations)



In [5]:
def store_conversation(session_id, user_message, ai_response):
    """유저 ID별 세션을 구분하여 대화 내역 저장"""

    timestamp = datetime.datetime.now().isoformat()

    # 유저 메시지 저장 (session_id 포함)
    user_doc = Document(
        page_content=user_message,
        metadata={"session_id": session_id, "role": "user", "timestamp": timestamp}
    )

    # AI 응답 저장 (session_id 포함)
    ai_doc = Document(
        page_content=ai_response,
        metadata={"session_id": session_id, "role": "assistant", "timestamp": timestamp}
    )

    # 벡터 저장소에 추가
    vector_store.add_documents([user_doc, ai_doc])

In [6]:
vector_store.get()

{'ids': ['c6dae5eb-3653-4264-8770-06334efa0462',
  'df4bed3a-173b-41da-8ad6-a2282e8d8b90',
  'dd2d2665-92fc-4c0b-b8c3-e20780619315',
  '072c9e2a-88b4-4905-9407-d7b775d968af',
  '31f4edac-2734-4ccc-afcb-d5b0988ad01a',
  'fde53191-f010-4ffc-97c3-11ada0f32b0e',
  '21a9696e-dbae-43f2-af90-b3d7591eac7c',
  '428ea681-f4cf-4380-a37b-ad7026de9398',
  '03f813c2-4562-4b50-8429-65cb30def3fb',
  '1f89918b-14cc-4b0f-8a19-c4a9d0139973',
  'f0c41d32-54f2-4ccd-b605-310b67826d6a',
  'e800a6a4-d175-4c43-a7ed-cff4e8257341',
  'c78fa6e8-6c53-48da-820b-7c5c92a2bc41',
  '1b8edaf0-da2b-4d03-8267-f59ec01dcf4b',
  '153e1bdc-9594-48f2-a8a1-c4e8f022a158',
  'c1a52b9d-e240-4991-afc6-5f8d305bdd39',
  '5a87f7e1-7b4f-4554-89f9-792fbe833cd5',
  'c3ab1515-3eaa-4173-87bc-05d95f6f300a',
  '2f30bfb4-9e38-4fc6-a21d-bfd3e8656625',
  '88fb8e77-9ac9-4e10-8f17-16fd94ef7a2c',
  '3befdc91-542a-40f5-9a93-51f3d73b94e5',
  '8d5554eb-2739-4ea3-9de3-40c1a7c6c628',
  'b036e0e2-71c1-4669-ace3-8c09e30ee38e',
  'e26aafb5-f0b2-4219-b16c-

## 2. Retriever 설정

In [59]:
session_id = "2"

vector_retriever = vector_store.as_retriever(search_type="mmr",search_kwargs={"k": 3, 'lambda_mult':0.1, "filter": {"session_id": session_id}})

## 3. prompt 구성

In [60]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate

examples = [
    {"input": "아이스 아메리카노 한 잔 주세요.", "answer": "ICE 아메리카노 한 잔 맞으시죠? 드시고 가시나요? 가지고 가시나요?"},
    {"input": "카페라떼 한 잔 주세요.", "answer": "ICE로 하실까요? HOT으로 하실까요?"},
    {"input": "너 몇 살이니? ", "answer": "주문과 상관없는 질문은 대답해드릴 수 없습니다. 다른 질문을 해주시겠어요?"},
    {"input": "가지고 가겠습니다.", "answer": "감사합니다. 결제 수단을 선택해주세요."}
]

example_prompt = PromptTemplate(
    input_variables= ["input","answer"],
    template="입력: {input} \n 출력: {answer} \n"
    )

Few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt = example_prompt,
    prefix ="""You are a helper agent who is built into the kiosk and tries to take orders.  
    Please be courteous and helpful to customers' questions and orders.
    Use the following pieces of retrieved tools to answer the question.
    If you don't know the answer, just say that you don't know.
    However, please only answer questions related to cafe work, such as inquiries about orders and menus.
    Answer in Korean.
    """, #프롬프트의 시작 부분
    suffix="고객: {input} \n AI: {agent_scratchpad}", #사용자 입력이 들어갈 부분
    input_variables=["input",'agent_scratchpad']
)

In [61]:
# prompt = PromptTemplate.from_template(
#     """
#     You are a helper agent who is built into the kiosk and tries to take orders.  
#     Please be courteous and helpful to customers' questions and orders.
#     Use the tools to answer the question.
#     If you don't know the answer, just say that you don't know.
#     However, please only answer questions related to cafe work, such as inquiries about orders and menus.
#     Answer in Korean.
    
#     고객 : {input}
    
#     tools : {agent_scratchpad}

#     답변 :
#     """

# )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helper agent who is built into the kiosk and tries to take orders."
            "Please be courteous and helpful to customers' questions and orders. "
            "You should check out their beverage options like hot and iced."
            "You should check out their other options like payment method."
            "However, please only answer questions related to cafe work, such as inquiries about orders and menus."

            "The ordering process is as follows: Select menu - Confirm takeout - Confirm additional menus or inquiries -Final confirmation of order with price information -Select payment method - Wait for payment - After payment, guidance message and 'Thank_you' is sent."

            "Always use the provided tools to answer."
            "Use chat_retriever_tool for questions related to order history. "
            "Answer all questions about orders and menus using menu_db_sql_tool."
            "In particular, please use menu_db_sql_tool to answer about price information."
            "you must answer in conversational form. Without any decoration such as symbols or dictionary forms."
            "Do not guess; retrieve data using the tools before responding."
            "Don't say you don't have it, just use the tools we provide."
            "Nnevertheless ,you don't know the answer, just say that you don't know."
            "Always reference the provided data when generating a response. Do not fabricate or infer any information."
            "Answer in Korean.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad', 'chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

# 4. llm 설정

In [62]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

chain = prompt | llm

# chain.invoke("몇 살이야?")

# 5. Agnet 설정 및 tool 설정

In [70]:
from langchain.agents import initialize_agent, AgentType
from langchain.agents import create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_sql_agent
from langchain_experimental.sql import SQLDatabaseChain
from langchain.sql_database import SQLDatabase
import sqlite3
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
import re
## tools custom

from langchain.tools.retriever import create_retriever_tool

chat_retriever_tool = create_retriever_tool(
    vector_retriever,    
    name='chat_search',
    description = 'use this tool to search chat history and order history'
)

# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri("sqlite:///../data/Comfile_Coffee_DB.db")

sql_llm =ChatOpenAI(model='gpt-4o-mini',temperature=0.2)

# SQLDatabaseChain 생성



def make_query_and_run(text: str):
    db = SQLDatabase.from_uri("sqlite:///../data/Comfile_Coffee_DB.db")
    info = db.get_table_info()

    custom_prompt = PromptTemplate(
    input_variables=["query"],
    partial_variables={'info':info},
    template="""
    You are an expert in SQL and databases. Generate an SQL query based on the given question. 

    - The question may not always provide the exact menu name, so extract only the key characteristics.
    - If multiple keywords are provided, process them using `AND`.
    - Use `LIKE` and `%` to handle partial matches.
    - If the question includes "price" or "가격", do not return only the price—return all related information by using `SELECT *`.
    - Convert "따뜻한" or "뜨거운" to "HOT" and "차가운" or "아이스" to "ICE" in SQL queries.

    Return only the raw SQL query without any markdown formatting, code blocks, or explanations.

    You must reference the following database structure:
    {info}



    question: {query}
    """
    )

    sql_chain = custom_prompt | sql_llm
    result_query=sql_chain.invoke(text).content
    result = re.sub(r"```sql\n(.*?)\n```", r"\1", result_query, flags=re.DOTALL).strip()
    return db.run(result)
    

menu_db_sql_tool = Tool(
    name="menu_db_sql_tool",
    func=make_query_and_run,
    description="You can search order information from Database"
)


tools= [menu_db_sql_tool, chat_retriever_tool]

agent = create_tool_calling_agent(llm=llm,tools=tools,prompt=prompt)

# agent = initialize_agent(
#     tools=[sql_tool],
#     llm=llm,
#    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#    verbose=True,
#      agent_kwargs={"prompt":prompt}
#  )

In [71]:
from langchain.agents import AgentExecutor

agent_exe=AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=8,
    must_use_tools= [menu_db_sql_tool]
)
result = agent_exe.invoke({"input":"아이스로 하나 주세요!"})
print(result['output'])



> Entering new AgentExecutor chain...
아이스 음료를 원하시는군요! 어떤 음료를 선택하시겠어요? 메뉴를 확인해 드릴게요.

> Finished chain.
아이스 음료를 원하시는군요! 어떤 음료를 선택하시겠어요? 메뉴를 확인해 드릴게요.


In [72]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_exe,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [75]:
user_input ="마시던 거로 주세요"

response = agent_with_chat_history.invoke(
    {"input": user_input},
    # session_id 설정
    config={"configurable": {"session_id": session_id}},
)

response['output']



> Entering new AgentExecutor chain...

Invoking: `chat_search` with `{'query': '마시던'}`


카페라떼는 2,900원입니다.

카페라떼는 2,900원입니다.

카페라떼 가격이 얼마인가요?마시던 음료는 카페라떼로 확인되었습니다. 카페라떼의 가격은 2,900원입니다. 이 메뉴로 주문하시겠어요?

> Finished chain.


'마시던 음료는 카페라떼로 확인되었습니다. 카페라떼의 가격은 2,900원입니다. 이 메뉴로 주문하시겠어요?'

In [18]:
store  

{'1': InMemoryChatMessageHistory(messages=[HumanMessage(content='가장 최근에 주문한 음료 주세요', additional_kwargs={}, response_metadata={}), AIMessage(content='가장 최근에 주문하신 음료는 아이스 카페라떼와 딸기라떼입니다. 가격은 다음과 같습니다:\n\n- 아이스 카페라떼: 2,900원\n- 딸기라떼: 3,500원\n\n총 합계는 6,400원입니다. 테이크아웃으로 진행할까요?', additional_kwargs={}, response_metadata={})])}

In [19]:
# 벡터 DB에 세션 데이터 저장
def save_session_to_vector_db(session_id):
    if session_id in store:
        chat_history = store[session_id].messages  # 현재 세션의 대화 내용 가져오기
        for msg in chat_history:
            vector_store.add_texts([msg.content], metadatas=[{"session_id": session_id, "role": msg.type}])
        print(f"✅ Session {session_id} data saved to vector DB.")

# 현재 활성화된 세션 초기화
def clear_session(session_id):
    if session_id in store:
        del store[session_id]  # 세션 데이터 삭제
        print(f"✅ Session {session_id} cleared.")

In [21]:
save_session_to_vector_db(session_id)
clear_session(session_id)

✅ Session 1 data saved to vector DB.
✅ Session 1 cleared.


In [66]:
vector_store.get(where={"session_id": session_id})

{'ids': ['59a01ff1-6aa7-41a3-9078-c9011a0aa1f5',
  'c6d10195-ae9f-4f08-a3e6-49ec4d08920b',
  '3ffc3fb5-efb9-473f-90c5-73d8af9d64da',
  '83faf17c-50d6-4265-80af-c069da450b3d'],
 'embeddings': None,
 'documents': ['카페라떼 가격이 얼마인가요?',
  '카페라떼는 2,900원입니다.',
  '카페라떼 가격이 얼마인가요?',
  '카페라떼는 2,900원입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'content': '카페라떼 가격이 얼마인가요?',
   'role': 'user',
   'session_id': '2',
   'timestamp': '2025-03-28T11:07:58.764313'},
  {'content': '카페라떼는 2,900원입니다.',
   'role': 'assistant',
   'session_id': '2',
   'timestamp': '2025-03-28T11:07:58.764313'},
  {'content': '카페라떼 가격이 얼마인가요?',
   'role': 'user',
   'session_id': '2',
   'timestamp': '2025-03-28T11:08:11.369707'},
  {'content': '카페라떼는 2,900원입니다.',
   'role': 'assistant',
   'session_id': '2',
   'timestamp': '2025-03-28T11:08:11.369707'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}